In [1]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu

augmented_data = pd.read_csv("/home/irteam/minhah-dcloud-dir/minhah/dreaddit/dreaddit_combined_data_gpt_more_11352_original.csv")
# 
augmented_data

,subreddit,text,label,intent,ilabels
0,ptsd,"""Feeling Overwhelmed by the Dreadful 'Sense o...",1,AnxietyandStressDisorders,0
1,ptsd,"""Desperately Seeking Relief from the Constant...",1,AnxietyandStressDisorders,0
2,ptsd,"""Feeling Overwhelmed by 'Doom' and Health Anx...",1,AnxietyandStressDisorders,0
3,assistance,Title - Seeking STEM Students' Assistance in ...,0,HousingSupportandAssistance,1
4,assistance,Title - Calling all STEM students for valuabl...,0,HousingSupportandAssistance,1
...,...,...,...,...,...
11347,relationships,"* Her, a week ago: Precious, how are you? (I i...",0,RelationshipsandAbuse,2
11348,ptsd,I don't have the ability to cope with it anymo...,1,AnxietyandStressDisorders,0
11349,anxiety,In case this is the first time you're reading ...,0,AnxietyandStressDisorders,0
11350,almosthomeless,Do you find this normal? They have a good rela...,0,HousingSupportandAssistance,1


In [2]:
# Load original and augmented data from CSV files
original_data = pd.read_csv("/home/irteam/minhah-dcloud-dir/minhah/dreaddit/dreaddit-train-pp.csv")
# augmented_data = pd.read_csv("/home/irteam/minhah-dcloud-dir/minhah/dreaddit/dreaddit-train-pp.csv")

# Merge the two dataframes with an outer join using `concat`.
merged_df = pd.concat([original_data, augmented_data], ignore_index=True)

# Identify and keep only the non-duplicate rows from the merged dataframe.
augmented_data = merged_df.drop_duplicates(keep=False)

# Reset the index of the resulting dataframe.
augmented_data.reset_index(drop=True, inplace=True)

# Create lists to store tokenized text
original_texts = original_data['text'].tolist()
augmented_texts = augmented_data['text'].tolist()

# Tokenize and convert to lowercase for original texts
original_texts = [nltk.word_tokenize(text.lower()) for text in original_texts]

# Tokenize and convert to lowercase for augmented texts
augmented_texts = [nltk.word_tokenize(str(text).lower()) for text in augmented_texts]

# Compute BLEU score for each pair of original and augmented texts
bleu_scores = []

for original, augmented in zip(original_texts, augmented_texts):
    score = sentence_bleu([original], augmented, weights=(0.5, 0.5))  # Using n=2 (bigrams)
    bleu_scores.append(score)

# Calculate the average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

print(f"Average BLEU Score: {average_bleu_score:.2f}")


/home/irteam/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Average BLEU Score: 0.05


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
augmented_data = pd.read_csv("/home/irteam/minhah-dcloud-dir/minhah/dreaddit/dreaddit_combined_data_gpt_more_11352_original.csv")
augmented_data
# Load original and augmented data from CSV files
original_data = pd.read_csv("/home/irteam/minhah-dcloud-dir/minhah/dreaddit/dreaddit-train-pp.csv")
# Merge the two dataframes with an outer join using `concat`.
merged_df = pd.concat([original_data, augmented_data], ignore_index=True)

# Identify and keep only the non-duplicate rows from the merged dataframe.
augmented_data = merged_df.drop_duplicates(keep=False)

# Reset the index of the resulting dataframe.
augmented_data.reset_index(drop=True, inplace=True)
# Assuming you have a 'text' column in both datasets
original_text = original_data['text']
# print(augmented_data['text'])
augmented_text = augmented_data['text'].astype(str)

# Convert text data to TF-IDF vectors
vectorizer = TfidfVectorizer()
original_tfidf = vectorizer.fit_transform(original_text)
augmented_tfidf = vectorizer.transform(augmented_text)

# Perform a t-test
t_stat, p_value = stats.ttest_ind(original_tfidf.toarray(), augmented_tfidf.toarray())

# Set your significance level (alpha)
alpha = 0.05

# Print the t-statistic and p-value
print(f'T-statistic: {t_stat[0]:.2f}')
print(f'P-value: {p_value[0]:.6f}')

# Check if any p-value is less than alpha
if (p_value < alpha).any():
    print("At least one comparison is statistically significant (reject the null hypothesis)")
else:
    print("No comparison is statistically significant (fail to reject the null hypothesis)")


T-statistic: 2.07
P-value: 0.038410
At least one comparison is statistically significant (reject the null hypothesis)
